In [8]:
# libraries to install
# !pip install langdetect
# !pip install selenium

In [9]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
import pandas as pd

In [10]:
# selenium with Chrome
driver = webdriver.Chrome('/Users/dany/Desktop/adm-hw3/chromedriver')

<ipython-input-10-caa2157e66d9>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/dany/Desktop/adm-hw3/chromedriver')


In [11]:
# create a dataframe with links of each anime
df = pd.DataFrame(columns = ['Href'])

I've gotten the idea from https://github.com/GiorgiaSalvatori/ADM-HW3/blob/main/main.ipynb. Also the following link was useful https://towardsdatascience.com/how-to-use-selenium-to-web-scrape-with-example-80f9b23a843a.

In [12]:
# go page by page and and store links in a list
anime_list = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime_list.append((link.contents[0], link.get('href')) )

100%|██████████| 400/400 [06:05<00:00,  1.09it/s]


In [13]:
print(len(anime_list))

19122


In [14]:
# assign list to dataframe
df['Href'] = anime_list

In [15]:
# Ok no duplicates
df['Href'].nunique()

19122

In [16]:
#save dataframe
df.to_csv('urls.csv')

We could also create a dictionary

In [20]:
#keys
name = []   
#values
url = []    

for item in anime_list:
    name.append(item[0])
    url.append(item[1])
    
D = dict(zip(name, url))
print(list(D.values())[0])

https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood
